In [45]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import import_roi
from utils import export_image_as_grid

# roi = import_roi('br_amazon')   .filter(ee.Filter.neq("id", 18413))
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()
first_year = 1985
last_year = 2020

## Biomass
Get mean biomass for edge pixels

In [46]:
biomass_1 = ee.Image("projects/ee-ana-zonia/assets/biomass_2020")
biomass = ee.Image("projects/ee-ana-zonia/assets/BIOMASS2").clip(roi)

# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age).int16().rename('agbd')

vis = {
    'min': 0,
    'max': 420,
    'palette': ['blue', 'red'],
}
   
Map = geemap.Map()
Map.addLayer(biomass.geometry(), {})
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Mature Forests

In [44]:
ecoregions = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi) \
                .map(lambda feature: feature.intersection(roi))

lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
  .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte()
# how many meters around a secondary forest is considered a "neighborhood" for mature forest calculations
# mat_neighbor = 1000

mature_mask = lulc.eq(3)
# # Mask the image to keep only pixels with the value 3 in all bands
mature_mask = mature_mask.reduce(ee.Reducer.allNonZero()).selfMask()

mature_biomass = biomass.updateMask(mature_mask).int16().rename('mat_biomass')
# # # Compute the median biomass values for mature forest per ecoregion.
# median_mature = mature_biomass.reduceRegions(ecoregions, reducer=ee.Reducer.median(), \
#                                              scale = 10000, crs = mature_biomass.projection().crs())

# # Convert the FeatureCollection to an image.
# median_mature = median_mature.reduceToImage(['mat_biomass'], ee.Reducer.first())

# Define the export parameters
export_params = {
    'image': mature_biomass,
    'description': 'mature_biomass',
    'assetId': 'projects/ee-ana-zonia/assets/mature_biomass_1',
    'scale': 10000,
    'crs': 'EPSG:4326'
}

# Create the export task
task = ee.batch.Export.image.toAsset(**export_params)

# Start the export task
task.start()

# frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020').updateMask(age).rename('frag')

# Compute sum over round kernel
# assuming that 99.7% of seeds come from forests within mat_neighbor distance, that's 3 standard deviations
# sd = mat_neighbor/3
# kernel = ee.Kernel.gaussian(radius = mat_neighbor, sigma = sd, units='meters', normalize = True)
# weighed_sum_mature = mature_biomass.convolve(kernel).rename('weighted_sum_mature')
# weighed_sum_mature

